In [0]:
import re
import matplotlib.pyplot as plot
import pandas as pd
from keras import Sequential
from keras.constraints import maxnorm
from keras.layers import Embedding, Conv1D, Dropout, MaxPooling1D, Flatten, Dense, LSTM, SpatialDropout1D
from keras.optimizers import SGD
from keras.utils import to_categorical
from keras_preprocessing.sequence import pad_sequences
from keras_preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [0]:
train_movie_df = pd.read_csv('train.tsv', delimiter='\t', encoding='utf-8')
test_movie_df = pd.read_csv('test.tsv', delimiter='\t', encoding='utf-8')

In [0]:
train_movie_df = train_movie_df.drop(columns=['PhraseId', 'SentenceId'])
label=train_movie_df[['Sentiment']]
train_movie_df=train_movie_df.drop(columns=['Sentiment'])
test_movie_df = test_movie_df.drop(columns=['PhraseId', 'SentenceId'])

In [0]:
train_movie_df['Phrase'] = train_movie_df['Phrase'].apply(lambda x: re.sub('[^a-zA-z0-9\s]', '', x.lower()))
test_movie_df['Phrase'] = test_movie_df['Phrase'].apply(lambda x: re.sub('[^a-zA-z0-9\s]', '', x.lower()))

In [0]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(train_movie_df['Phrase'].values)
X_train = tokenizer.texts_to_sequences(train_movie_df['Phrase'].values)
X_train = pad_sequences(X_train)

In [48]:
X_train.shape

(156060, 45)

In [0]:

max_fatures = 1000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(test_movie_df['Phrase'].values)
X_test = tokenizer.texts_to_sequences(test_movie_df['Phrase'].values)
X_test = pad_sequences(X_test)

In [50]:
X_test.shape

(66292, 45)

In [0]:
embed_dim = 20
lstm_out = 10  

In [0]:
def create_model():
    model = Sequential()
    model.add(Embedding(13734, embed_dim, input_length=X_train.shape[1]))
    # model.add(SpatialDropout1D(0.4))
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(5, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [58]:
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(label)
Y_train = to_categorical(integer_encoded)
X_tr, X_te, Y_tr, Y_te = train_test_split(X_train, Y_train, test_size=0.25, random_state=30)
print(X_tr.shape,Y_tr.shape)
print(X_te.shape,Y_te.shape)

(117045, 45) (117045, 5)
(39015, 45) (39015, 5)


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:251: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [0]:
model = create_model()

In [60]:
history = model.fit(X_tr, Y_tr, epochs=5, batch_size=40, validation_data=(X_te,Y_te))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 117045 samples, validate on 39015 samples
Epoch 1/5
117045/117045 [==============================] - 264s 2ms/step - loss: 1.0471 - accuracy: 0.5865 - val_loss: 0.9809 - val_accuracy: 0.6103
Epoch 2/5
117045/117045 [==============================] - 262s 2ms/step - loss: 0.9511 - accuracy: 0.6248 - val_loss: 0.9563 - val_accuracy: 0.6212
Epoch 3/5
117045/117045 [==============================] - 258s 2ms/step - loss: 0.9163 - accuracy: 0.6370 - val_loss: 0.9423 - val_accuracy: 0.6281
Epoch 4/5
117045/117045 [==============================] - 259s 2ms/step - loss: 0.8907 - accuracy: 0.6471 - val_loss: 0.9324 - val_accuracy: 0.6318
Epoch 5/5
117045/117045 [==============================] - 257s 2ms/step - loss: 0.8677 - accuracy: 0.6577 - val_loss: 0.9308 - val_accuracy: 0.6356


PREDICTION ON TEST DATA SET

In [61]:
y_pred=model.predict_classes(X_test[:1])
print(y_pred[0]," PREDICTED LABEL")

2  PREDICTED LABEL


In [62]:
sub_file = pd.read_csv('sampleSubmission.csv',sep=',')
print(sub_file['Sentiment'].iloc[0]," ACTUAL LABEL")

2  ACTUAL LABEL


QUESTION 6 : Changing HyperParameters

In [63]:
from keras.optimizers import adam
embed_dim=50
lstm_out=50
s=adam(lr=0.001)
model1 = Sequential()
model1.add(Embedding(13734, embed_dim, input_length = X_train.shape[1]))
model1.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model1.add(Dense(Y_train.shape[1],activation='softmax'))
model1.compile(loss = 'binary_crossentropy', optimizer=s,metrics = ['accuracy'])
print(model1.summary())

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 45, 50)            686700    
_________________________________________________________________
lstm_4 (LSTM)                (None, 50)                20200     
_________________________________________________________________
dense_4 (Dense)              (None, 5)                 255       
Total params: 707,155
Trainable params: 707,155
Non-trainable params: 0
_________________________________________________________________
None


In [64]:
history1 = model1.fit(X_tr, Y_tr, epochs=5, batch_size=50, validation_data=(X_te,Y_te))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 117045 samples, validate on 39015 samples
Epoch 1/5
117045/117045 [==============================] - 207s 2ms/step - loss: 0.3582 - accuracy: 0.8432 - val_loss: 0.3356 - val_accuracy: 0.8518
Epoch 2/5
117045/117045 [==============================] - 207s 2ms/step - loss: 0.3306 - accuracy: 0.8536 - val_loss: 0.3300 - val_accuracy: 0.8534
Epoch 3/5
117045/117045 [==============================] - 206s 2ms/step - loss: 0.3234 - accuracy: 0.8572 - val_loss: 0.3272 - val_accuracy: 0.8544
Epoch 4/5
117045/117045 [==============================] - 207s 2ms/step - loss: 0.3179 - accuracy: 0.8599 - val_loss: 0.3251 - val_accuracy: 0.8556
Epoch 5/5
117045/117045 [==============================] - 207s 2ms/step - loss: 0.3132 - accuracy: 0.8621 - val_loss: 0.3233 - val_accuracy: 0.8566
